# Top 10 Finishers Prediction

Below are the columns required for the supervised learning model. Columns marked with an asterisk (*) are indices and won't be included in the model input. Columns marked with a hash (#) are labels.

- **Index Columns:**
  - *RaceId {year-session} `[race_id]`
  - *RacerId {firstname-lastname} `[racer_id]`
  - *TrackId {circuitname} `[track_id]`

- **Feature Columns:**
  - Type of track `[is_street]`
    - 0 for purpose-built track
    - 1 for street track
  - Previous year result for racer `[prev_year_pos]` (0 if not available or didn't participate) 
  - Qualifying position `[qualifying_pos]` 
  - Qualifying timing:
    - Q1 `[q1_timing]` 
    - Q2 `[q2_timing]` (if racer didn't qualify for Q2, this will be 0) 
    - Q3 `[q3_timing]` (if racer didn't qualify for Q3, this will be 0) 

- **Label Column:**
  - #Race finish `[race_finish]` 

## Imports and Configuration

In [1]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Standard library imports
import sys
import os
import logging
import time
import json

# Third-party imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from joblib import dump, load

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# User-defined imports
module_path = os.path.abspath(os.path.join("..", "scripts"))
if module_path not in sys.path:
    sys.path.append(module_path)

from utilities import load_and_process_csv, join_dataframes

: 

In [2]:
# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Constants
REQUIRED_COLUMNS = ["driverId", "q1", "q2", "q3", "race_id"]

# Load Data

In [3]:
# Load and process data
race_information_filepath = "../data/raw/Race_Information_1995_2023.csv"
race_results_filepath = "../data/raw/Race_Results_1995_2023.csv"
qualifying_results_filepath = "../data/raw/Qualification_Results_1995_2023.csv"

# load data
race_information = load_and_process_csv(race_information_filepath, ["season", "round"])
race_results = load_and_process_csv(
    race_results_filepath,
    ["season", "round"],
    rename_columns={"grid": "qualifying_pos",
                    "url": "driver_wiki_url",
                    "nationality": "driver_nationality",
                    "url.1": "constructor_wiki_url",
                    "nationality.1": "constructor_nationality"
                    },
)


qualifying_results = load_and_process_csv(
    qualifying_results_filepath,
    ["season", "round"],
    rename_columns={"q1": "q1_timing", "q2": "q2_timing", "q3": "q3_timing"},
    fill_na_value=0,
)

# join race information with results
race_results_information = join_dataframes(race_results, race_information, "race_id")

# display information
display(race_results_information.info())
display(qualifying_results.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11435 entries, 0 to 11434
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   number                   11435 non-null  int64  
 1   position                 11435 non-null  int64  
 2   positionText             11435 non-null  object 
 3   points                   11435 non-null  float64
 4   qualifying_pos           11435 non-null  int64  
 5   laps                     11435 non-null  int64  
 6   status                   11435 non-null  object 
 7   season_x                 11435 non-null  int64  
 8   round_x                  11435 non-null  int64  
 9   FastestLap               7805 non-null   object 
 10  driverId                 11435 non-null  object 
 11  code                     9574 non-null   object 
 12  driver_wiki_url          11435 non-null  object 
 13  givenName                11435 non-null  object 
 14  familyName            

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9623 entries, 0 to 9622
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   season         9623 non-null   int64 
 1   round          9623 non-null   int64 
 2   driverId       9623 non-null   object
 3   driver         9623 non-null   object
 4   constructorId  9623 non-null   object
 5   q1_timing      9623 non-null   object
 6   q2_timing      9623 non-null   object
 7   q3_timing      9623 non-null   object
 8   race_id        9623 non-null   object
dtypes: int64(2), object(7)
memory usage: 676.7+ KB


None

In 2004, F1 started 

In [17]:
race_results['status'].value_counts()

status
Finished         4896
+1 Lap           2403
+2 Laps           636
Collision         518
Engine            462
                 ... 
Oil pump            1
+12 Laps            1
Tyre puncture       1
+26 Laps            1
+17 Laps            1
Name: count, Length: 109, dtype: int64

In [18]:
race_results

,number,position,positionText,points,qualifying_pos,laps,status,season,round,FastestLap,...,dateOfBirth,driver_nationality,permanentNumber,constructorId,constructor_wiki_url,name,constructor_nationality,millis,time,race_id
0,1,1,1,10.0,2,71,Finished,1995,1,NaN,...,1969-01-03,German,NaN,benetton,http://en.wikipedia.org/wiki/Benetton_Formula,Benetton,Italian,5914154.0,1:38:34.154,1995_1
1,6,2,2,6.0,3,71,Finished,1995,1,NaN,...,1971-03-27,British,NaN,williams,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Williams,British,5925214.0,+11.060,1995_1
2,28,3,3,4.0,5,70,+1 Lap,1995,1,NaN,...,1959-08-27,Austrian,NaN,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian,NaN,NaN,1995_1
3,8,4,4,3.0,7,70,+1 Lap,1995,1,NaN,...,1968-09-28,Finnish,NaN,mclaren,http://en.wikipedia.org/wiki/McLaren,McLaren,British,NaN,NaN,1995_1
4,27,5,5,2.0,6,70,+1 Lap,1995,1,NaN,...,1964-06-11,French,NaN,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian,NaN,NaN,1995_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11430,2,16,16,0.0,20,58,Finished,2023,22,"{'rank': '12', 'lap': '43', 'Time': {'time': '...",...,2000-12-31,American,2.0,williams,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,Williams,British,5310415.0,+1:27.791,2023_22
11431,24,17,17,0.0,19,58,Finished,2023,22,"{'rank': '13', 'lap': '43', 'Time': {'time': '...",...,1999-05-30,Chinese,24.0,alfa,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Alfa Romeo,Swiss,5312046.0,+1:29.422,2023_22
11432,55,18,18,0.0,16,57,Retired,2023,22,"{'rank': '17', 'lap': '42', 'Time': {'time': '...",...,1994-09-01,Spanish,55.0,ferrari,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Ferrari,Italian,NaN,NaN,2023_22
11433,77,19,19,0.0,18,57,+1 Lap,2023,22,"{'rank': '18', 'lap': '42', 'Time': {'time': '...",...,1989-08-28,Finnish,77.0,alfa,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Alfa Romeo,Swiss,NaN,NaN,2023_22


In [ ]:
# Assuming you want to keep all matches and all columns from both DataFrames are relevant
merged_df = pd.merge(
    REQUIRED_COLUMNS,
    race_results_information[
        [
            "driverId",
            "race_id",
            "position",
            "qualifying_pos",
            "prev_year_pos",
            "season",
            "round",
        ]
    ],
    on=["driverId", "race_id"],
    how="inner",
    validate="one_to_one",
)

# Directly saving the required DataFrame to CSV, no need to create an intermediate DataFrame
merged_df.to_csv("../data/prepared/Complete_data_supervised_learning.csv", index=False)

In [ ]:
# You can also read the data directly (if available)
final_data = pd.read_csv("Complete_data_supervised_learning.csv")

In [ ]:
# Convert timing columns to seconds
final_data['q1_timing'] = final_data['q1_timing'].apply(convert_to_seconds)
final_data['q2_timing'] = final_data['q2_timing'].apply(convert_to_seconds)
final_data['q3_timing'] = final_data['q3_timing'].apply(convert_to_seconds)
final_data['position'] = final_data['position'].apply(lambda x: 1 if x <= 10 else 0)
final_data_required = final_data[["q1_timing", "q2_timing", "q3_timing", "qualifying_pos", "prev_year_pos", "position"]]
final_data_scaled = final_data_required.copy()

In [ ]:
scaler = MinMaxScaler()
final_data_scaled[['q1_timing', 'q2_timing', 'q3_timing', 'qualifying_pos', 'prev_year_pos']] = scaler.fit_transform(final_data_scaled[['q1_timing', 'q2_timing', 'q3_timing', 'qualifying_pos', 'prev_year_pos']])

# Calculate correlation matrix
corr_matrix = final_data_scaled.corr()

# Plotting the correlation matrix as a heatmap
plt.figure(figsize=(20, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Splitting the dataset into features and the label
label_col = "position"
X = final_data_required.drop(label_col, axis=1)
y = final_data_required[label_col]

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizing the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Modeling

### Linear Methods

In [ ]:
def build_best_logistic_regression_model(
    X_train_scaled, X_test_scaled, y_train, y_test, param_grid, **grid_search_kwargs
):
    """
    Builds and returns the best Logistic Regression model using GridSearchCV.

    Args:
        X_train_scaled (pd.DataFrame): Scaled training features.
        X_test_scaled (pd.DataFrame): Scaled test features.
        y_train (pd.Series): Training labels.
        y_test (pd.Series): Test labels.
        param_grid (dict): Parameter grid for GridSearchCV.
        **grid_search_kwargs: Additional keyword arguments for GridSearchCV.

    Returns:
        best_model (LogisticRegression): The best Logistic Regression model.
        accuracy (float): Accuracy of the best model on the test set.
    """
    # Initializing the Logistic Regression model
    log_reg = LogisticRegression(random_state=42, max_iter=1000)

    # Setting up GridSearchCV to find the best model
    grid_search = GridSearchCV(
        estimator=log_reg,
        param_grid=param_grid,
        cv=5,
        scoring="accuracy",
        n_jobs=-1,
        verbose=1,
        **grid_search_kwargs,
    )

    start_time = time.time()
    logging.info("Starting GridSearchCV to find the best Logistic Regression model.")

    try:
        # Fitting GridSearchCV to the training data
        grid_search.fit(X_train_scaled, y_train)

        # Extracting the best estimator (model)
        best_model = grid_search.best_estimator_

        # Making predictions with the best model on the test set
        y_pred = best_model.predict(X_test_scaled)

        # Calculating the accuracy of the best model
        accuracy = accuracy_score(y_test, y_pred)

        end_time = time.time()
        elapsed_time = end_time - start_time

        logging.info(f"GridSearchCV completed in {elapsed_time:.2f} seconds.")
        logging.info(f"Best Model's Accuracy: {accuracy * 100:.2f}%")
        logging.info(f"Best Parameters: {grid_search.best_params_}")

        # Returning the best model and its accuracy
        return best_model, accuracy

    except Exception as e:
        logging.error(f"An error occurred during GridSearchCV: {e}")
        return None, None


# Defining the parameter grid for logistic regression
lr_param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],
    "penalty": ["l1", "l2"],
    "solver": ["liblinear"],  # 'liblinear' is compatible with l1 and l2 penalties.
}

# Building and evaluating the Logistic Regression model
best_lr_model, best_lr_accuracy = build_best_logistic_regression_model(
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    lr_param_grid,
    return_train_score=True,
)

if best_lr_model is not None:
    dump(best_lr_model, "best_logistic_regression_model.joblib")
    # You can also read the model directly if available
    # best_lr_model = load("best_logistic_regression_model.joblib")

### Ensemble Methods

In [ ]:
def build_best_random_forest_model(
    X_train, X_test, y_train, y_test, param_grid, **grid_search_kwargs
):
    """
    Builds and returns the best Random Forest model using GridSearchCV.

    Args:
        X_train (pd.DataFrame): Training features.
        X_test (pd.DataFrame): Test features.
        y_train (pd.Series): Training labels.
        y_test (pd.Series): Test labels.
        param_grid (dict): Parameter grid for GridSearchCV.
        **grid_search_kwargs: Additional keyword arguments for GridSearchCV.

    Returns:
        best_model (RandomForestClassifier): The best Random Forest model.
        accuracy (float): Accuracy of the best model on the test set.
    """
    # Initializing the Random Forest classifier
    rf_clf = RandomForestClassifier(random_state=42)

    # Setting up GridSearchCV to find the best model
    grid_search = GridSearchCV(
        estimator=rf_clf,
        param_grid=param_grid,
        cv=5,
        scoring="accuracy",
        n_jobs=-1,
        verbose=1,
        **grid_search_kwargs,
    )

    start_time = time.time()
    logging.info("Starting GridSearchCV to find the best Random Forest model.")

    try:
        # Fitting GridSearchCV to the training data
        grid_search.fit(X_train, y_train)

        # Extracting the best estimator (model)
        best_model = grid_search.best_estimator_

        # Making predictions with the best model on the test set
        y_pred = best_model.predict(X_test)

        # Calculating the accuracy of the best model
        accuracy = accuracy_score(y_test, y_pred)

        end_time = time.time()
        elapsed_time = end_time - start_time

        logging.info(f"GridSearchCV completed in {elapsed_time:.2f} seconds.")
        logging.info(f"Best Model's Accuracy: {accuracy * 100:.2f}%")
        logging.info(f"Best Parameters: {grid_search.best_params_}")

        # Returning the best model and its accuracy
        return best_model, accuracy

    except Exception as e:
        logging.error(f"An error occurred during GridSearchCV: {e}")
        return None, None


# Defining the parameter grid for Random Forest
rf_param_grid = {
    "n_estimators": [10, 50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
}

# Example usage with additional GridSearchCV parameters
best_rf_model, best_rf_accuracy = build_best_random_forest_model(
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    rf_param_grid,
    return_train_score=True,
)

if best_rf_model is not None:
    dump(best_rf_model, "best_random_forest_model.joblib")

    # You can also read the model directly if available
    # best_rf_model = load("../models/best_random_forest_model.joblib")

### Neural Network Methods

In [ ]:
def create_model(hidden_nodes=1, learning_rate=0.001):
    """
    Creates and compiles a Keras Sequential model.

    Args:
        hidden_nodes (int): Number of nodes in the hidden layer.
        learning_rate (float): Learning rate for the optimizer.

    Returns:
        model (Sequential): Compiled Keras model.
    """
    # Define the model
    model = Sequential()
    model.add(Dense(hidden_nodes, input_dim=X_train_scaled.shape[1], activation="relu"))
    model.add(Dense(1, activation="sigmoid"))  # Output layer for binary classification

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model


def build_best_neural_network_model(
    X_train_scaled, X_test_scaled, y_train, y_test, param_grid, **grid_search_kwargs
):
    """
    Builds and returns the best neural network model using GridSearchCV.

    Args:
        X_train_scaled (pd.DataFrame): Scaled training features.
        X_test_scaled (pd.DataFrame): Scaled test features.
        y_train (pd.Series): Training labels.
        y_test (pd.Series): Test labels.
        param_grid (dict): Parameter grid for GridSearchCV.
        **grid_search_kwargs: Additional keyword arguments for GridSearchCV.

    Returns:
        best_model (KerasClassifier): The best neural network model.
        accuracy (float): Accuracy of the best model on the test set.
    """
    # Wrapping the Keras model so it can be used by scikit-learn
    model = KerasClassifier(build_fn=create_model, verbose=0)

    # Setting up GridSearchCV to find the best model
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,
        scoring="accuracy",
        n_jobs=-1,
        verbose=1,
        **grid_search_kwargs,
    )

    start_time = time.time()
    logging.info("Starting GridSearchCV to find the best neural network model.")

    try:
        # Fitting GridSearchCV to the training data
        grid_search.fit(X_train_scaled, y_train)

        # Extracting the best estimator (model)
        best_model = grid_search.best_estimator_

        # Making predictions with the best model on the test set
        y_pred = best_model.predict(X_test_scaled)
        y_pred = np.round(y_pred).astype(int)  # Convert probabilities to binary output

        # Calculating the accuracy of the best model
        accuracy = accuracy_score(y_test, y_pred)

        end_time = time.time()
        elapsed_time = end_time - start_time

        logging.info(f"GridSearchCV completed in {elapsed_time:.2f} seconds.")
        logging.info(f"Best Model's Accuracy: {accuracy * 100:.2f}%")
        logging.info(f"Best Parameters: {grid_search.best_params_}")

        # Returning the best model and its accuracy
        return best_model, accuracy

    except Exception as e:
        logging.error(f"An error occurred during GridSearchCV: {e}")
        return None, None


# Defining the parameter grid for the neural network
nn_param_grid = {
    "hidden_nodes": [10, 50, 100],
    "learning_rate": [0.001, 0.01, 0.1],
    "epochs": [50],
    "batch_size": [32],
}

# Building and evaluating the neural network model
best_nn_model, best_nn_accuracy = build_best_neural_network_model(
    X_train_scaled,
    X_test_scaled,
    y_train,
    y_test,
    nn_param_grid,
    return_train_score=True,
)

if best_nn_model is not None:
    dump(best_nn_model, "best_neural_network_model.joblib")

    # You can also read the model directly if available
    # best_nn_model = load("../models/best_neural_network_model.joblib")

### Model Evaluation

In [ ]:
def perform_sensitivity_analysis(
    models, X_scaled, feature_names, output_dir=".", file_format="png"
):
    """
    Perform sensitivity analysis on given models.

    Args:
        models (dict): Dictionary of trained models with their names as keys.
        X_scaled (np.ndarray): Scaled features array.
        feature_names (list): List of feature names.
        output_dir (str): Directory to save the sensitivity analysis plots.
        file_format (str): File format for saving plots.
    """
    # Number of points to evaluate for each feature
    num_points = 100

    # Results dictionary to store sensitivity data
    sensitivity_results = {}

    # Iterate through each model
    for model_name, model in models.items():
        logging.info(f"Analyzing model: {model_name}")
        sensitivity_results[model_name] = {}

        # Iterate through each feature
        for feature in feature_names:
            # Array to hold predictions
            predictions = []

            # Generate values across the range
            feature_index = feature_names.index(feature)
            min_val, max_val = np.min(X_scaled[:, feature_index]), np.max(
                X_scaled[:, feature_index]
            )
            values = np.linspace(min_val, max_val, num_points)

            # Modify one feature at a time, keeping others constant
            for val in values:
                X_temp = np.copy(X_scaled)
                X_temp[:, feature_index] = val
                try:
                    pred = model.predict(X_temp)
                    predictions.append(np.mean(pred))
                except Exception as e:
                    logging.error(
                        f"Error predicting with model {model_name} for feature {feature}: {e}"
                    )
                    predictions.append(np.nan)  # Use NaN to indicate prediction failure

            sensitivity_results[model_name][feature] = np.nanstd(
                predictions
            )  # Use nanstd to handle NaNs

    # Plotting the sensitivity results
    for model_name, sensitivities in sensitivity_results.items():
        plt.figure(figsize=(10, 6))
        plt.title(f"Sensitivity Analysis for {model_name}")
        plt.bar(range(len(sensitivities)), list(sensitivities.values()), align="center")
        plt.xticks(
            range(len(sensitivities)), list(sensitivities.keys()), rotation="vertical"
        )
        plt.ylabel("Standard Deviation of Predictions")

        file_path = f"{output_dir}/sensitivity_{model_name}.{file_format}"
        plt.savefig(file_path, dpi=300)
        plt.show()
        logging.info(f"Sensitivity plot saved to {file_path}")


# Example usage
models = {
    "Random Forest": best_rf_model,
    "Logistic Regression": best_lr_model,
    "Neural Network": best_nn_model,
}

feature_names = [
    "q1_timing",
    "q2_timing",
    "q3_timing",
    "qualifying_pos",
    "prev_year_pos",
]
perform_sensitivity_analysis(models, X_test_scaled, feature_names)

In [ ]:
# Assuming final_data_required is your DataFrame
sns.pairplot(final_data_required, hue='position')
plt.savefig('pairplot.png', dpi=300)
plt.show()

### Visualizations

In [ ]:

# Assuming X_train and y_train are already defined
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)

# Getting feature importances
feature_importances = rf.feature_importances_

# Converting to a DataFrame for easier plotting
importances_df = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances})

# Sorting by importance
importances_df = importances_df.sort_values('importance', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=importances_df)
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()